## Отчет по исследованию Looped Transformers и их возможностей.
Мини исследование-отчет по статье - https://arxiv.org/abs/2311.12424

### Сравнение эффективности Looped TFs с обычными TFs
Все эксперементы проводились с выключенным Mixed Precision, по скольку он не влиял на скорость обучения - видимо из-за того что gpt2_nano не может использовать GPU эффективно.
Одна часть эксперементов проводилась на локальной машине с Nvidia 3060ti mobile, а вторая на удаленной машине с Nvidia Tesla P100.

Посмотрим на скорость сходимости трансформера (TF) с параметром $ L = 12 $ и Looped TFs с параметрами $ L = 1; b = \{5, 10, 20, 25\}$. Были использованы следующие параметры Heads=4, dims=10, points=31. Потери масштабированы на размерность регрессии.
<br />
<p float="left" align="center">
    <img src="../images/Compare_losses_b_n.png" alt="isolated" width="700" height="350"/>
</p>

Как можно видеть из графиков, Looped TFs при увеличении $ b $ дает улучшение метрик. При количестве параметров в 12 раз меньшим, чем у обычного трансформера, он показывает неплохие показатели. Конфигурации для запуска моделей взяты из папки эксперимента №5.

### Итеративные свойства
По скольку в статье подразумевается что Looped TFs имеют итеративные свойства, мы можем их проверить на примере линейной регрессии.
Для этого обучим Looped TFs со значениями $ b =/{5, 10, 20/} $ и посмотрим на их сходимость при больших $ b $.
<br />
<p float="left" align="center">
    <img src="../images/check_for_convergence_properties.png" alt="isolated" width="700" height="450"/>
</p>

Из графика можем видеть что при большем $ b $ на этапе валидации Looped TF сходится при $ b > 5 $


In [ ]:
Указать блокнот с кодом эксперимента

### Попробуем брать только N последних токенов
Чтобы понять на сколько мы можем обрезать количество токенов которые подаются модель замаскируем часть из них маской и посмотрим на метрики.


## Looped n-layers.


In [13]:
from models import TransformerModelLoopedLastNTokens
from train import get_task_sampler
transformer_model = TransformerModelLoopedLastNTokens(
    n_dims=2,
    n_positions=101,
    n = 2,
    n_embd=4,
    n_layer=1,
    n_head=2,
    pred_type="regression",
).cuda()

number of parameters: 0.00M


In [16]:
task_sampler = get_task_sampler(
    task_name="linear_regression",
    batch_size=2,
    n_points=31,
    n_dims=2,
    n_dims_truncated=3,
    device="cuda"
)

real_task = task_sampler()
xs, ys = real_task.xs.float(), real_task.ys.float()
n_loops = 3  # K
n_loop_window = 20

horizon_start = max(0, n_loops - n_loop_window)
## forward pass 
B, n, d_in = xs.shape
zs = transformer_model._combine(xs, ys)  # [B, n, d_in], [B, n], [B, n] -> [B, 2n, d_in + 1]

In [17]:
transformer_model(xs, ys, horizon_start, n_loops)

RuntimeError: The size of tensor a (118) must match the size of tensor b (62) at non-singleton dimension 1

In [18]:
zs

tensor([[[-7.3075e-01,  6.0446e-01],
         [-1.9309e+00,  0.0000e+00],
         [-2.7123e-01, -7.0409e-02],
         [-3.0348e-01,  0.0000e+00],
         [-9.3926e-01, -4.0257e-04],
         [-1.3922e+00,  0.0000e+00],
         [-4.1362e-01, -1.0511e+00],
         [ 8.6011e-01,  0.0000e+00],
         [ 1.8804e-01,  1.7921e-02],
         [ 2.5370e-01,  0.0000e+00],
         [ 1.1649e+00,  5.2976e-01],
         [ 9.8465e-01,  0.0000e+00],
         [ 5.7636e-01,  8.0791e-01],
         [-2.7792e-01,  0.0000e+00],
         [-5.9148e-01,  7.4665e-01],
         [-1.9237e+00,  0.0000e+00],
         [ 7.0331e-01,  1.1026e-01],
         [ 8.8831e-01,  0.0000e+00],
         [-1.4756e-01, -7.8835e-01],
         [ 8.8633e-01,  0.0000e+00],
         [ 8.7242e-02, -2.4755e-01],
         [ 4.7638e-01,  0.0000e+00],
         [ 8.8326e-01, -3.6956e-01],
         [ 1.8278e+00,  0.0000e+00],
         [ 5.4087e-01,  1.5845e+00],
         [-1.4192e+00,  0.0000e+00],
         [-1.1003e+00,  1.0457e-01],
 

In [19]:
transformer_model._read_in(zs), transformer_model._read_in(zs).shape

(tensor([[[ 5.9070e-01, -1.1256e+00, -1.4359e-01,  1.1622e-01],
          [ 1.0074e+00, -1.6976e+00,  8.2762e-01,  5.5047e-01],
          [-1.6677e-01, -6.2492e-01, -7.4551e-02, -3.8640e-02],
          [-9.8401e-02, -6.6721e-01, -9.0607e-02, -2.8061e-02],
          [ 3.3332e-01, -1.0696e+00,  2.6831e-01,  1.9796e-01],
          [ 6.4133e-01, -1.3565e+00,  5.2366e-01,  3.5896e-01],
          [-7.1701e-01, -4.1036e-01,  4.8287e-01,  2.4315e-02],
          [-8.8901e-01,  6.9517e-02, -7.4713e-01, -4.4170e-01],
          [-4.2054e-01, -3.6158e-01, -3.7665e-01, -2.0301e-01],
          [-4.7698e-01, -3.1443e-01, -4.0498e-01, -2.2613e-01],
          [-7.4658e-01,  9.7870e-02, -1.1768e+00, -5.5670e-01],
          [-9.7364e-01,  1.4837e-01, -8.1740e-01, -4.8597e-01],
          [-1.6320e-01, -3.6118e-01, -9.8007e-01, -3.5099e-01],
          [-1.1577e-01, -6.5103e-01, -1.0503e-01, -3.7148e-02],
          [ 5.8988e-01, -1.0816e+00, -2.9134e-01,  6.4928e-02],
          [ 1.0025e+00, -1.6931e+00,  8.

In [20]:
transformer_model.get_last_n_tokens(transformer_model._read_in(zs), 2)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.